In [2]:
# Import the dependencies
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Klyuchi,RU,2021-05-26 17:26:25,52.2667,79.1667,82.76,22,24,8.28
1,1,Aklavik,CA,2021-05-26 17:26:26,68.2191,-135.0107,34.52,74,90,13.80
2,2,Jamestown,US,2021-05-26 17:22:14,42.0970,-79.2353,80.02,61,40,7.00
3,3,Albany,US,2021-05-26 17:26:21,42.6001,-73.9662,89.28,52,41,3.00
4,4,New Norfolk,AU,2021-05-26 17:26:26,-42.7826,147.0587,46.65,65,98,1.99


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [22]:
# Heatmap of temp
# Get the lat and lng
locations = city_data_df[['Lat','Lng']]
# Get the max temp
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    if temp >= 0:
        temps.append(temp)
# Assign the figure
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer
fig.add_layer(heat_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Heatmap of percent Humidity
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Heatmap of percent cloudiness
locations = city_data_df[['Lat','Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
locations = city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0,30.1), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Ask the customer to add a min and max temp value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

In [33]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Klyuchi,RU,2021-05-26 17:26:25,52.2667,79.1667,82.76,22,24,8.28
2,2,Jamestown,US,2021-05-26 17:22:14,42.0970,-79.2353,80.02,61,40,7.00
3,3,Albany,US,2021-05-26 17:26:21,42.6001,-73.9662,89.28,52,41,3.00
7,7,Manggar,ID,2021-05-26 17:26:27,-2.8833,108.2667,83.25,77,83,7.94
15,15,Rikitea,PF,2021-05-26 17:26:29,-23.1203,-134.9692,76.71,79,38,25.12
21,21,Hilo,US,2021-05-26 17:26:30,19.7297,-155.0900,75.25,79,90,5.75
28,28,Atuona,PF,2021-05-26 17:23:39,-9.8000,-139.0333,78.89,73,33,11.88
35,35,Victoria,HK,2021-05-26 17:25:01,22.2855,114.1577,86.32,84,68,1.99
39,39,Bengkulu,ID,2021-05-26 17:24:05,-3.8004,102.2655,78.10,84,100,4.63
50,50,Kokopo,PG,2021-05-26 17:26:37,-4.3520,152.2633,75.79,91,100,5.59


In [35]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = city_data_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Klyuchi,RU,82.76,52.2667,79.1667,
1,Aklavik,CA,34.52,68.2191,-135.0107,
2,Jamestown,US,80.02,42.0970,-79.2353,
3,Albany,US,89.28,42.6001,-73.9662,
4,New Norfolk,AU,46.65,-42.7826,147.0587,


In [37]:
# Set parameters to search for a hotel
params = {
    'radius': 5000,
    'type':'lodging',
    'key': g_key
}

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']

    # Add the latitude and longitude to location key for the params dictionary
    params['location'] = f'{lat},{lng}'

    # Use the search term: 'lodging' and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [38]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Klyuchi,RU,82.76,52.2667,79.1667,Vesolaya Mayovka
1,Aklavik,CA,34.52,68.2191,-135.0107,Aklavik Inn
2,Jamestown,US,80.02,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,Albany,US,89.28,42.6001,-73.9662,
4,New Norfolk,AU,46.65,-42.7826,147.0587,The Shingles Riverside Cottages
...,...,...,...,...,...,...
558,Zeya,RU,40.42,53.7500,127.2667,Kapital Inn
559,Naron,ES,63.68,43.5167,-8.1528,Marcial
560,Leh,IN,46.71,34.1667,77.5833,Hotel Ladakh Greens
561,Kalmunai,LK,83.12,7.4167,81.8167,VS Villa


In [43]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [44]:
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [45]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[['Lat','Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))